In [ ]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, Bidirectional
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

print(clean_sentence("I will surf today !!"))

i will surf today 


In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
dutch_data = ["Hoeveel kost dit?", "Ik mag je erg graag.", "Maak hem alsjeblieft wakker.", "Waar is het vliegveld?", "Het boek is te duur.", "Het heeft geen zin.", "Ik zou graag iets willen kopen.", "De TV is kapot.", "Het is niet te moeilijk.", "Dat is niet genoeg."]
german_data = ["Wie viel kostet das?", "Ich mag Sie sehr", "Bitte weckt ihn auf.", "Wo ist der Flughafen?", "Das Buch ist zu teuer.", "Das macht keinen Sinn.", "Ich würde gerne etwas kaufen.", "Der Fernseher ist kaputt.", "Es ist nicht allzu schwierig.", "Das ist nicht genug."]
english_data = ["How much does this cost?", "I like you very much.", "Please wake him up.", "Where is the airport?", "The book is too expensive.", "It makes no sense.", "I would like to buy something.", "The TV is broken.", "It is not too difficult.", "That is not enough."]

In [ ]:
# Clean sentences
english_sentences = [clean_sentence(tmp) for tmp in english_data]
german_sentences = [clean_sentence(tmp) for tmp in german_data]
dutch_sentences = [clean_sentence(tmp) for tmp in dutch_data]

print(english_sentences[0])

how much does this cost


In [ ]:
# Tokenize words
english_text_tokenized, english_text_tokenizer = tokenize(english_sentences)
german_text_tokenized, german_text_tokenizer = tokenize(german_sentences)
dutch_text_tokenized, dutch_text_tokenizer = tokenize(dutch_sentences)

print(english_text_tokenized)
print(english_text_tokenizer)

# Check language length
english_vocab = len(english_text_tokenizer.word_index) + 1
print("English vocabulary is of {} unique words".format(english_vocab))

german_vocab = len(german_text_tokenizer.word_index) + 1
print("german vocabulary is of {} unique words".format(german_vocab))

dutch_vocab = len(dutch_text_tokenizer.word_index) + 1
print("dutch vocabulary is of {} unique words".format(dutch_vocab))

[[9, 3, 10, 11, 12], [4, 5, 13, 14, 3], [15, 16, 17, 18], [19, 1, 2, 20], [2, 21, 1, 6, 22], [7, 23, 24, 25], [4, 26, 5, 27, 28, 29], [2, 30, 1, 31], [7, 1, 8, 6, 32], [33, 1, 8, 34]]
English vocabulary is of 35 unique words
german vocabulary is of 34 unique words
dutch vocabulary is of 34 unique words


In [ ]:
max_sentence_length = 6
english_pad_sentence = pad_sequences(english_text_tokenized, max_sentence_length, padding = "post")
german_pad_sentence = pad_sequences(german_text_tokenized, max_sentence_length, padding = "post")
dutch_pad_sentence = pad_sequences(dutch_text_tokenized, max_sentence_length, padding = "post")

# Reshape data
english_pad_sentence = english_pad_sentence.reshape(*english_pad_sentence.shape, 1)
german_pad_sentence = german_pad_sentence.reshape(*german_pad_sentence.shape, 1)
dutch_pad_sentence = dutch_pad_sentence.reshape(*dutch_pad_sentence.shape, 1)

In [ ]:
input_shape = (max_sentence_length, 2)
input_sequence = Input(input_shape, name='InputLayer')
rnn = LSTM(256, return_sequences=True, dropout=0.5, name='RNNLayer')(input_sequence)
logits = TimeDistributed(Dense(dutch_vocab), name='TimeDistributed')(rnn)

model = Model(input_sequence, Activation('softmax')(logits))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-2),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      [(None, 6, 2)]            0         
_________________________________________________________________
RNNLayer (LSTM)              (None, 6, 256)            265216    
_________________________________________________________________
TimeDistributed (TimeDistrib (None, 6, 34)             8738      
_________________________________________________________________
activation (Activation)      (None, 6, 34)             0         
Total params: 273,954
Trainable params: 273,954
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(english_pad_sentence[0].shape)
print(english_pad_sentence[0])

print(german_pad_sentence[0].shape)
print(german_pad_sentence[0])

(6, 1)
[[ 9]
 [ 3]
 [10]
 [11]
 [12]
 [ 0]]
(6, 1)
[[6]
 [7]
 [8]
 [2]
 [0]
 [0]]


In [ ]:
eng_ger_data = np.array([np.concatenate((english_pad_sentence[i], german_pad_sentence[i]), axis=1) for i in range(len(english_pad_sentence))])
print(eng_ger_data.shape)
print(eng_ger_data[0])

(10, 6, 2)
[[ 9  6]
 [ 3  7]
 [10  8]
 [11  2]
 [12  0]
 [ 0  0]]


In [ ]:
model_results = model.fit(eng_ger_data, dutch_pad_sentence, batch_size=1, epochs=100)


Epoch 1/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3055 - accuracy: 0.9000
Epoch 2/100
10/10 [==============================] - 0s 3ms/step - loss: 0.0803 - accuracy: 0.9667
Epoch 3/100
10/10 [==============================] - 0s 3ms/step - loss: 0.2562 - accuracy: 0.9333
Epoch 4/100
10/10 [==============================] - 0s 3ms/step - loss: 0.7628 - accuracy: 0.7500
Epoch 5/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3455 - accuracy: 0.8500
Epoch 6/100
10/10 [==============================] - 0s 3ms/step - loss: 0.5302 - accuracy: 0.8833
Epoch 7/100
10/10 [==============================] - 0s 3ms/step - loss: 0.4232 - accuracy: 0.8333
Epoch 8/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3134 - accuracy: 0.8667
Epoch 9/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3864 - accuracy: 0.8667
Epoch 10/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3485 - accuracy: 0.8667
Epoch 11/

In [ ]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
index = 4
print("The english sentence is: {}".format(english_sentences[index]))
print("The german sentence is: {}".format(german_sentences[index]))
print("The dutch sentence is    : {}".format(dutch_sentences[index]))
print('The predicted sentence is: ', end="")
print(logits_to_sentence(model.predict(eng_ger_data[index:index+1])[0], dutch_text_tokenizer))

The english sentence is: the book is too expensive
The german sentence is: das buch ist zu teuer
The dutch sentence is    : het boek is te duur
The predicted sentence is: het boek is te duur <empty>
